In [1]:
import pandas as pd
# one hot encoding
# check if we have cyclical values
# Check scaling
# Check features that are correlated
# Pycaret for choosing the best models, pick top 5 and then tune their hyperparameters. 

# hyperparameter-tuning: random? bayesian optimisation? grid-search takes a lot of time but is precise
    # Optuna
    # Hyper-opt
df = pd.read_csv("diabetes_Data.csv")
df.isnull()

,Age,Sex,HighChol,CholCheck,BMI,Smoker,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,GenHlth,MentHlth,PhysHlth,DiffWalk,Stroke,HighBP,Diabetes
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70687,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
70688,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
70689,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
70690,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
